# Tosho Recommender (Hyperparam, Architecture Search)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import random
import fasttext as ft
import pickle
import optuna
import gensim
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from model import *

In [4]:
np.random.seed(1)
tf.random.set_seed(1)

## Load Data

In [5]:
text_hist_pd = pd.read_csv('data/text_hist_perm.csv.bz', sep='\t', compression='bz2')
text_hist_pd.shape

(33600, 5)

In [6]:
text_hist_pd.sample(3, random_state=1)

,book1,book2,book3,book4,label
17402,深い 霧 の たちこめる 漆 の 海 姿 なき 幽霊 船長 ゴースト キャプテン が 舵 を...,アイスダンス ペア 樹里 と 恵 は 初 出場 し 界 選手権 大会 で 大 喝采 を 浴び...,そろそろ 考え ちゃ 悩め ブタ シッタカブッタ は 人生 の 見え 方 を ていねい に ...,なぞなぞ 大好き の 妖怪 たち が 読者 に おもしろい なぞなぞ で いどみ 人気 の ...,1700233
9348,宗教 と は 国 と は 人間 へ の 深い まなざし が 捉え 青春 小説 の 新 境地 ...,わたし じゃ よ わ ー ん わ ー ん 学校 から の 帰り道 大きな 声 て い 清香 ...,担任 に 納得 の いか 絵 を 描か さ れ ユク は 傷つく 憧れ の 少女 に 励まさ...,ひな まつり に 愛花 の 家 で は お ばあちゃん だけ が つくれる ごちそう 肉まん...,1902265
32139,夜 の 星空 を 見 げた とき に 学校 で 習っ 星座 が 星座 の 名前 の 通り に...,界 の カエル と 日本 の カエル 200 種類 を カラー 写真 で 紹介 分布 や す...,現在 もっとも 謎 の 多い チョウ モンシロチョウ 最新 の 撮影 技術 と 研究 成果 ...,ほ 類 は虫類 ペット 家 ちく アニマル ウォッチング 絶滅 危機 動物 とじこみ 付録 ...,1400121


## Sample 20% for Hyperparam Tuning

In [26]:
text_hist_sample_pd = text_hist_pd.sample(frac=.2, random_state=1)
text_hist_sample_pd.shape

(6720, 5)

## Split into Train and Val

In [27]:
train_texts, val_texts, y_train, y_val = train_test_split(text_hist_sample_pd.drop(['label'], axis=1),
                                                          text_hist_sample_pd[['label']].values,
                                                          test_size = .2,
                                                          random_state=1234)

## Tokenize

In [28]:
# Limit on the number of word features to the top 20K features.
TOP_K = 20000

# Most claims doc are within 10K word length
MAX_SEQUENCE_LENGTH = 500

def sequence_vectorize(train_texts, y_train, val_texts, y_val):
    # Create vocabulary with training texts.
    tokenizer = Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts['book1'])

    # Vectorize training and validation texts.
    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
    x_trn_1 = pad_sequences(tokenizer.texts_to_sequences(train_texts['book1']), maxlen=MAX_SEQUENCE_LENGTH)
    x_trn_2 = pad_sequences(tokenizer.texts_to_sequences(train_texts['book2']), maxlen=MAX_SEQUENCE_LENGTH)
    x_trn_3 = pad_sequences(tokenizer.texts_to_sequences(train_texts['book3']), maxlen=MAX_SEQUENCE_LENGTH)
    x_trn_4 = pad_sequences(tokenizer.texts_to_sequences(train_texts['book4']), maxlen=MAX_SEQUENCE_LENGTH)
    x_val_1 = pad_sequences(tokenizer.texts_to_sequences(val_texts['book1']), maxlen=MAX_SEQUENCE_LENGTH)
    x_val_2 = pad_sequences(tokenizer.texts_to_sequences(val_texts['book2']), maxlen=MAX_SEQUENCE_LENGTH)
    x_val_3 = pad_sequences(tokenizer.texts_to_sequences(val_texts['book3']), maxlen=MAX_SEQUENCE_LENGTH)
    x_val_4 = pad_sequences(tokenizer.texts_to_sequences(val_texts['book4']), maxlen=MAX_SEQUENCE_LENGTH)
    
    return x_trn_1, x_trn_2, x_trn_3, x_trn_4, x_val_1, x_val_2, x_val_3, x_val_4, tokenizer.word_index, tokenizer

In [29]:
x_trn_1, x_trn_2, x_trn_3, x_trn_4, x_val_1, x_val_2, x_val_3, x_val_4, word_index, tokenizer = sequence_vectorize(train_texts, y_train, val_texts, y_val)

In [30]:
# number of vocabulary in the corpus
len(word_index)

13474

### Load FastText (Japanese)

In [31]:
ja_vecs = ft.load_model('model/cc.ja.300.bin')

In [32]:
ja_vecs.get_word_vector('あ').shape

(300,)

In [33]:
EMBEDDING_DIM = 300

## Load Book2Vec Vectors

In [34]:
b2v = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format('model/book2vec', binary=False)

/opt/conda/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning:

This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function



## Use Book2Vec vectors as label

In [35]:
y_train_gru = []
for book_id in y_train:
    y_train_gru.append(b2v.get_vector(str(book_id[0])))

y_train_gru = np.array(y_train_gru)

In [36]:
y_val_gru = []
for book_id in y_val:
    y_val_gru.append(b2v.get_vector(str(book_id[0])))

y_val_gru = np.array(y_val_gru)

In [37]:
y_train.shape, y_train_gru.shape, y_val.shape, y_val_gru.shape

((5376, 1), (5376, 392), (1344, 1), (1344, 392))

## Model

### Embedding layer

In [38]:
embedding_matrix = np.zeros((TOP_K+1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i < TOP_K:
        try:
            embedding_vector = ja_vecs.get_word_vector(word)
        except:
            embedding_vector = np.zeros(EMBEDDING_DIM)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [39]:
max_sequence = x_trn_1.shape[1] if MAX_SEQUENCE_LENGTH > x_trn_1.shape[1] else MAX_SEQUENCE_LENGTH

### Optuna for Hyperparameter Tuning

In [40]:
def objective(trial):
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    rnn_units = trial.suggest_int("rnn_unit", 150, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 3e-2)
    share_gru_weights_on_book = trial.suggest_categorical('share_gru_weights_on_book', [True, False])
    use_attention_on_book = trial.suggest_categorical('use_attention_on_book', [True, False])
    use_attention_on_user = trial.suggest_categorical('use_attention_on_user', [True, False])
    use_batch_norm = trial.suggest_categorical('use_batch_norm', [True, False])
    is_embedding_trainable = trial.suggest_categorical('is_embedding_trainable', [True, False])
    final_activation = trial.suggest_categorical('final_activation', ['tanh', 'relu'])
    
    model = gru_model(
                     embedding_dim=EMBEDDING_DIM,
                     dropout_rate=dropout_rate,
                     rnn_unit=rnn_units,
                     input_shape=(max_sequence,),
                     num_features=TOP_K+1,
                     share_gru_weights_on_book=share_gru_weights_on_book,
                     use_attention_on_book=use_attention_on_book,
                     use_attention_on_user=use_attention_on_user,
                     use_batch_norm=use_batch_norm,
                     is_embedding_trainable=is_embedding_trainable,
                     final_activation=final_activation,
                     final_dimension=b2v.vector_size,
                     embedding_matrix=embedding_matrix)

    # Compile model
    loss = 'cosine_similarity'
    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=['cosine_similarity'])

    history = model.fit(
        [x_trn_1, x_trn_2, x_trn_3, x_trn_4],
        y_train_gru,
        epochs=1,
        validation_data=([x_val_1, x_val_2, x_val_3, x_val_4], y_val_gru),
        batch_size=256)
    
    return -np.amax(history.history['val_cosine_similarity'])

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

Train on 5376 samples, validate on 1344 samples
5376/5376 [==============================] - 121s 23ms/sample - loss: -0.1396 - cosine_similarity: 0.1396 - val_loss: -0.3364 - val_cosine_similarity: 0.3364


[I 2019-10-05 18:47:32,512] Finished trial#0 resulted in value: -0.3363998830318451. Current best value is -0.3363998830318451 with parameters: {'dropout_rate': 0.3444406298376848, 'rnn_unit': 154, 'learning_rate': 0.0010006896385182243, 'share_gru_weights_on_book': True, 'use_attention_on_book': False, 'use_attention_on_user': False, 'use_batch_norm': True, 'is_embedding_trainable': False, 'final_activation': 'relu'}.


Train on 5376 samples, validate on 1344 samples
5376/5376 [==============================] - 117s 22ms/sample - loss: -0.4369 - cosine_similarity: 0.4369 - val_loss: -0.4667 - val_cosine_similarity: 0.4667


[I 2019-10-05 18:49:40,853] Finished trial#1 resulted in value: -0.4666786193847656. Current best value is -0.4666786193847656 with parameters: {'dropout_rate': 0.474022288123939, 'rnn_unit': 197, 'learning_rate': 0.009763053032493729, 'share_gru_weights_on_book': False, 'use_attention_on_book': True, 'use_attention_on_user': False, 'use_batch_norm': True, 'is_embedding_trainable': True, 'final_activation': 'tanh'}.


Train on 5376 samples, validate on 1344 samples
5376/5376 [==============================] - 127s 24ms/sample - loss: -0.3402 - cosine_similarity: 0.3402 - val_loss: -0.3711 - val_cosine_similarity: 0.3711


...

Train on 5376 samples, validate on 1344 samples
5376/5376 [==============================] - 124s 23ms/sample - loss: -0.4810 - cosine_similarity: 0.4810 - val_loss: -0.5242 - val_cosine_similarity: 0.5242


[I 2019-10-05 22:24:21,532] Finished trial#83 resulted in value: -0.5241971611976624. Current best value is -0.5263495445251465 with parameters: {'dropout_rate': 0.18886535264626358, 'rnn_unit': 196, 'learning_rate': 0.004263337782047885, 'share_gru_weights_on_book': True, 'use_attention_on_book': True, 'use_attention_on_user': True, 'use_batch_norm': False, 'is_embedding_trainable': False, 'final_activation': 'tanh'}.


Train on 5376 samples, validate on 1344 samples
5376/5376 [==============================] - 126s 23ms/sample - loss: -0.4792 - cosine_similarity: 0.4792 - val_loss: -0.5221 - val_cosine_similarity: 0.5221


[I 2019-10-05 22:27:31,830] Finished trial#84 resulted in value: -0.5220504999160767. Current best value is -0.5263495445251465 with parameters: {'dropout_rate': 0.18886535264626358, 'rnn_unit': 196, 'learning_rate': 0.004263337782047885, 'share_gru_weights_on_book': True, 'use_attention_on_book': True, 'use_attention_on_user': True, 'use_batch_norm': False, 'is_embedding_trainable': False, 'final_activation': 'tanh'}.


Train on 5376 samples, validate on 1344 samples
3840/5376 [====================>.........] - ETA: 32s - loss: -0.4614 - cosine_similarity: 0.4614

In [44]:
study.best_params

{'dropout_rate': 0.20894163487979106,
 'rnn_unit': 194,
 'learning_rate': 0.004383349360427826,
 'share_gru_weights_on_book': True,
 'use_attention_on_book': True,
 'use_attention_on_user': True,
 'use_batch_norm': False,
 'is_embedding_trainable': False,
 'final_activation': 'tanh'}

In [45]:
study.best_value

-0.526502788066864

In [46]:
trial_df = study.trials_dataframe()

In [47]:
trial_df.sort_values(by='value')

number                state     value             datetime_start  \
                                                                      
98     98  TrialState.COMPLETE -0.526503 2019-10-05 23:09:08.771033   
42     42  TrialState.COMPLETE -0.526350 2019-10-05 20:21:41.822816   
26     26  TrialState.COMPLETE -0.525876 2019-10-05 19:42:37.793860   
51     51  TrialState.COMPLETE -0.525051 2019-10-05 20:46:11.447992   
99     99  TrialState.COMPLETE -0.524974 2019-10-05 23:12:22.938098   
52     52  TrialState.COMPLETE -0.524885 2019-10-05 20:48:56.485860   
92     92  TrialState.COMPLETE -0.524845 2019-10-05 22:49:46.216634   
81     81  TrialState.COMPLETE -0.524628 2019-10-05 22:14:52.456390   
67     67  TrialState.COMPLETE -0.524596 2019-10-05 21:31:21.154080   
31     31  TrialState.COMPLETE -0.524514 2019-10-05 19:54:03.995780   
22     22  TrialState.COMPLETE -0.524437 2019-10-05 19:33:42.197807   
55     55  TrialState.COMPLETE -0.524356 2019-10-05 20:57:09.553774   
88     88  TrialState.COMPLETE -0.524309 2019-10-05 22:37:01.274261   
93     93  TrialState.COMPLETE -0.524228 2019-10-05 22:52:58.397893   
83     83  TrialState.COMPLETE -0.524197 2019-10-05 22:21:08.472252   
74     74  TrialState.COMPLETE -0.524137 2019-10-05 21:52:48.932273   
61     61  TrialState.COMPLETE -0.524106 2019-10-05 21:14:00.496107   
82     82  TrialState.COMPLETE -0.524066 2019-10-05 22:18:01.298860   
54     54  TrialState.COMPLETE -0.523750 2019-10-05 20:54:27.554784   
69     69  TrialState.COMPLETE -0.523640 2019-10-05 21:37:17.667116   
27     27  TrialState.COMPLETE -0.523576 2019-10-05 19:44:52.820310   
41     41  TrialState.COMPLETE -0.523491 2019-10-05 20:19:09.066147   
62     62  TrialState.COMPLETE -0.523465 2019-10-05 21:16:49.027375   
65     65  TrialState.COMPLETE -0.523380 2019-10-05 21:25:33.217380   
70     70  TrialState.COMPLETE -0.523257 2019-10-05 21:40:23.331363   
89     89  TrialState.COMPLETE -0.523169 2019-10-05 22:40:11.792032   
71     71  TrialState.COMPLETE -0.523153 2019-10-05 21:43:26.371951   
91     91  TrialState.COMPLETE -0.523079 2019-10-05 22:46:38.691022   
60     60  TrialState.COMPLETE -0.522974 2019-10-05 21:11:09.006199   
45     45  TrialState.COMPLETE -0.522944 2019-10-05 20:29:50.584823   
..    ...                  ...       ...                        ...   
95     95  TrialState.COMPLETE -0.498539 2019-10-05 22:59:22.459144   
56     56  TrialState.COMPLETE -0.495008 2019-10-05 20:59:53.071693   
68     68  TrialState.COMPLETE -0.492931 2019-10-05 21:34:18.551131   
39     39  TrialState.COMPLETE -0.489954 2019-10-05 20:14:01.036145   
49     49  TrialState.COMPLETE -0.487416 2019-10-05 20:40:43.693854   
28     28  TrialState.COMPLETE -0.487364 2019-10-05 19:47:09.223751   
3       3  TrialState.COMPLETE -0.482780 2019-10-05 18:52:00.295147   
48     48  TrialState.COMPLETE -0.476906 2019-10-05 20:37:55.147992   
36     36  TrialState.COMPLETE -0.475733 2019-10-05 20:06:07.254136   
34     34  TrialState.COMPLETE -0.475644 2019-10-05 20:01:11.918862   
38     38  TrialState.COMPLETE -0.471968 2019-10-05 20:11:17.807586   
57     57  TrialState.COMPLETE -0.468804 2019-10-05 21:02:39.625890   
1       1  TrialState.COMPLETE -0.466679 2019-10-05 18:47:32.516158   
90     90  TrialState.COMPLETE -0.466469 2019-10-05 22:43:25.603599   
17     17  TrialState.COMPLETE -0.462407 2019-10-05 19:22:53.116451   
25     25  TrialState.COMPLETE -0.452409 2019-10-05 19:40:23.748148   
20     20  TrialState.COMPLETE -0.447726 2019-10-05 19:29:19.459438   
10     10  TrialState.COMPLETE -0.446412 2019-10-05 19:07:37.791479   
58     58  TrialState.COMPLETE -0.417590 2019-10-05 21:05:26.158565   
8       8  TrialState.COMPLETE -0.374161 2019-10-05 19:03:04.469695   
2       2  TrialState.COMPLETE -0.371098 2019-10-05 18:49:40.856601   
77     77  TrialState.COMPLETE -0.370212 2019-10-05 22:02:13.440359   
50     50  TrialState.COMPLETE -0.369537 2019-10-05 20:43:29.612309   
37     37  TrialState.COMPL